In [18]:
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import folium
import time
import ipywidgets as widgets
from IPython.display import display, clear_output
import datetime

In [10]:
#pip install requests beautifulsoup4 folium geopy

     ---------------------------------------- 40.3/40.3 kB ? eta 0:00:00
   ---------------------------------------- 125.4/125.4 kB 1.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [22]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# Path to your WebDriver (e.g., ChromeDriver)
webdriver_path = 'D:/s9/Concertify/chromedriver-win64/chromedriver.exe'

# Initialize a Selenium WebDriver
driver = webdriver.Chrome(webdriver_path)

base_url = "https://www.ticketmaster.fr"

# Go to the Ticketmaster Paris page
driver.get('https://www.ticketmaster.fr/fr/paris')
time.sleep(5)  # Wait for JavaScript to load the content

# Get the page source and close the browser
html = driver.page_source
driver.quit()

# Parse the page with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

concerts = []
for i in range(20):
    event = soup.find('li', id=f'event-result-{i}')
    if event:
        title = event.find('h3', class_='event-result-title').get_text(strip=True)
        date = event.find('time', itemprop='startDate').get_text(strip=True)

        # Extract the arena name
        arena = event.find('span', itemprop='name').get_text(strip=True) if event.find('span', itemprop='name') else "Unknown Arena"

        # Extract the city location
        city_location = event.find('span', itemprop='addressLocality').get_text(strip=True) if event.find('span', itemprop='addressLocality') else "Unknown City"

        # Extract the event URL
        event_link = event.find('a', class_='event-result-title-link')
        url = base_url + event_link['href'] if event_link and event_link.has_attr('href') else "URL not found"
        
        genre_item = event.find('p', class_='event-result-genre-item')
        full_genre = genre_item.get_text(strip=True) if genre_item else "Unknown Genre"
        genre = full_genre.split(' - ')[-1] if ' - ' in full_genre else full_genre


        location = f"{arena}, {city_location}"
        concerts.append({"title": title, "location": location, "date": date, "event_url": url, "genre": genre})


# Print the concert information
for concert in concerts:
    print(concert)


{'title': 'KAROL G', 'location': 'ACCOR ARENA, PARIS', 'date': '22 juin 2024', 'event_url': 'https://www.ticketmaster.fr/fr/manifestation/karol-g-billet/idmanif/570897', 'genre': 'SALSA ET AFRO-CUBAIN'}
{'title': 'PACKAGE KAROL G', 'location': 'ACCOR ARENA, PARIS', 'date': '22 juin 2024', 'event_url': 'https://www.ticketmaster.fr/fr/pack/package-karol-g-billet/idpack/3356', 'genre': 'SALSA ET AFRO-CUBAIN'}
{'title': 'PACKAGE KAROL G', 'location': 'ACCOR ARENA, PARIS', 'date': '23 juin 2024', 'event_url': 'https://www.ticketmaster.fr/fr/pack/package-karol-g-billet/idpack/3385', 'genre': 'SALSA ET AFRO-CUBAIN'}
{'title': 'SLIPKNOT', 'location': 'ACCOR ARENA, PARIS', 'date': '12 décembre 2024', 'event_url': 'https://www.ticketmaster.fr/fr/manifestation/slipknot-billet/idmanif/571645', 'genre': 'HARD METAL'}
{'title': 'SLIPKNOT', 'location': 'ACCOR ARENA, PARIS', 'date': '12 décembre 2024', 'event_url': 'https://www.ticketmaster.fr/fr/pack/slipknot-billet/idpack/3366', 'genre': 'HARD METAL

In [28]:
# Function to convert French date strings to datetime objects
def translate_month_fr_to_en(date_str):
    fr_to_en_months = {
        'janvier': 'January',
        'février': 'February',
        'mars': 'March',
        'avril': 'April',
        'mai': 'May',
        'juin': 'June',
        'juillet': 'July',
        'août': 'August',
        'septembre': 'September',
        'octobre': 'October',
        'novembre': 'November',
        'décembre': 'December'
    }
    for fr, en in fr_to_en_months.items():
        if fr in date_str:
            return date_str.replace(fr, en)
    return date_str

def str_to_date(date_str):
    date_str = translate_month_fr_to_en(date_str)
    return datetime.datetime.strptime(date_str, '%d %B %Y').date()  # Convert to date object


# Function to geocode locations
def geocode_location(location):
    geolocator = Nominatim(user_agent="geoapiExercises")
    try:
        loc = geolocator.geocode(location)
        return (loc.latitude, loc.longitude)
    except:
        return None

# Geocode each concert location
for concert in concerts:
    coords = geocode_location(concert['location'])
    if coords:
        concert['coords'] = coords
    else:
        concert['coords'] = (0, 0)  # Default coords

# Create date range widgets for date filter
date_start_filter = widgets.DatePicker(description='Start Date')
date_end_filter = widgets.DatePicker(description='End Date')

# Create dropdown widget for genre filter
genre_filter = widgets.Dropdown(options=['All'] + sorted(set(concert['genre'] for concert in concerts)))

def filter_concerts(change):
    # Clear the current output
    clear_output(wait=True)
    display(date_start_filter, date_end_filter, genre_filter)

    # Filter concerts based on the selected date range and genre
    filtered_concerts = concerts
    if date_start_filter.value and date_end_filter.value:
        start_date = date_start_filter.value
        end_date = date_end_filter.value
        filtered_concerts = [concert for concert in filtered_concerts if start_date <= str_to_date(concert['date']) <= end_date]
    if genre_filter.value != 'All':
        filtered_concerts = [concert for concert in filtered_concerts if concert['genre'] == genre_filter.value]

    # Group filtered concerts by location
    filtered_concerts_by_location = {}
    for concert in filtered_concerts:
        key = concert['coords']
        if key not in filtered_concerts_by_location:
            filtered_concerts_by_location[key] = {'name': concert['location'], 'concerts': []}
        filtered_concerts_by_location[key]['concerts'].append(concert)

    # Create a new map with filtered concerts
    map = folium.Map(location=[48.8566, 2.3522], zoom_start=12)
    for location, data in filtered_concerts_by_location.items():
        location_name = data['name']
        concerts_at_location = data['concerts']

        popup_content = f"<h3>{location_name}</h3><ul>"
        for concert in concerts_at_location:
            # Create a hyperlink for each concert
            concert_link = f"<a href='{concert['event_url']}' target='_blank'>{concert['title']}</a>"
            popup_content += f"<li>{concert_link} - {concert['date']}</li>"
        popup_content += '</ul>'

        # Use BeautifulSoup to format HTML in the popup
        soup = BeautifulSoup(popup_content, 'html.parser')
        popup_content = str(soup)

        folium.Marker(
            location=location,
            popup=folium.Popup(popup_content, max_width=300),
            tooltip=location_name
        ).add_to(map)

    # Display the new filtered map
    display(map)

# Observe changes in the widgets and update the map accordingly
date_start_filter.observe(filter_concerts, names='value')
date_end_filter.observe(filter_concerts, names='value')


genre_filter.observe(filter_concerts, names='value')
reset_genre_button = widgets.Button(description='Reset Genre')

# Display the widgets and the initial map
display(date_start_filter, date_end_filter, genre_filter)
filter_concerts(None)

DatePicker(value=None, description='Start Date')

DatePicker(value=None, description='End Date')

Dropdown(options=('All', 'COMEDIE MUSICALE', 'DANSE HIP-HOP', 'HARD METAL', 'POP ROCK FOLK', 'RAP HIP-HOP SLAM…

In [31]:
# Function to convert French date strings to datetime objects
def translate_month_fr_to_en(date_str):
    fr_to_en_months = {
        'janvier': 'January',
        'février': 'February',
        'mars': 'March',
        'avril': 'April',
        'mai': 'May',
        'juin': 'June',
        'juillet': 'July',
        'août': 'August',
        'septembre': 'September',
        'octobre': 'October',
        'novembre': 'November',
        'décembre': 'December'
    }
    for fr, en in fr_to_en_months.items():
        if fr in date_str:
            return date_str.replace(fr, en)
    return date_str

def str_to_date(date_str):
    date_str = translate_month_fr_to_en(date_str)
    return datetime.datetime.strptime(date_str, '%d %B %Y').date()  # Convert to date object


# Function to geocode locations
def geocode_location(location):
    geolocator = Nominatim(user_agent="geoapiExercises")
    try:
        loc = geolocator.geocode(location)
        return (loc.latitude, loc.longitude)
    except:
        return None

# Geocode each concert location
for concert in concerts:
    coords = geocode_location(concert['location'])
    if coords:
        concert['coords'] = coords
    else:
        concert['coords'] = (0, 0)  # Default coords
        
# Create widgets for date, genre, and location filters
date_start_filter = widgets.DatePicker(description='Start Date')
date_end_filter = widgets.DatePicker(description='End Date')
genre_filter = widgets.Dropdown(options=['All'] + sorted(set(concert['genre'] for concert in concerts)))
location_filter = widgets.Dropdown(options=['All'] + sorted(set(concert['location'] for concert in concerts)))

def filter_concerts(change):
    # Clear the current output
    clear_output(wait=True)
    display(widgets.Label('Filter by Date:'), date_start_filter, date_end_filter)
    display(widgets.Label('Filter by Genre:'), genre_filter)
    display(widgets.Label('Filter by Location:'), location_filter)

    # Filter concerts based on the selected filters
    filtered_concerts = concerts
    if date_start_filter.value and date_end_filter.value:
        start_date = date_start_filter.value
        end_date = date_end_filter.value
        filtered_concerts = [concert for concert in filtered_concerts if start_date <= str_to_date(concert['date']) <= end_date]
    if genre_filter.value != 'All':
        filtered_concerts = [concert for concert in filtered_concerts if concert['genre'] == genre_filter.value]
    if location_filter.value != 'All':
        filtered_concerts = [concert for concert in filtered_concerts if concert['location'] == location_filter.value]

    # Group filtered concerts by location
    filtered_concerts_by_location = {}
    for concert in filtered_concerts:
        key = concert['coords']
        if key not in filtered_concerts_by_location:
            filtered_concerts_by_location[key] = {'name': concert['location'], 'concerts': []}
        filtered_concerts_by_location[key]['concerts'].append(concert)

    # Sort concerts by date within each location
    for location_data in filtered_concerts_by_location.values():
        location_data['concerts'].sort(key=lambda x: str_to_date(x['date']))

    # Create a new map with filtered concerts
    map = folium.Map(location=[48.8566, 2.3522], zoom_start=12)
    for location, data in filtered_concerts_by_location.items():
        location_name = data['name']
        concerts_at_location = data['concerts']

        popup_content = f"<h3>{location_name}</h3><ul>"
        for concert in concerts_at_location:
            # Create a hyperlink for each concert
            concert_link = f"<a href='{concert['event_url']}' target='_blank'>{concert['title']}</a>"
            popup_content += f"<li>{concert_link} - {concert['date']}</li>"
        popup_content += '</ul>'

        # Use BeautifulSoup to format HTML in the popup
        soup = BeautifulSoup(popup_content, 'html.parser')
        popup_content = str(soup)

        folium.Marker(
            location=location,
            popup=folium.Popup(popup_content, max_width=300),
            tooltip=location_name
        ).add_to(map)

    # Display the new filtered map
    display(map)

# Observe changes in the widgets and update the map accordingly
date_start_filter.observe(filter_concerts, names='value')
date_end_filter.observe(filter_concerts, names='value')
genre_filter.observe(filter_concerts, names='value')
location_filter.observe(filter_concerts, names='value')

# Display the widgets and the initial map
display(widgets.Label('Filter by Date:'), date_start_filter, date_end_filter)
display(widgets.Label('Filter by Genre:'), genre_filter)
display(widgets.Label('Filter by Location:'), location_filter)
filter_concerts(None)    


Label(value='Filter by Date:')

DatePicker(value=None, description='Start Date')

DatePicker(value=None, description='End Date')

Label(value='Filter by Genre:')

Dropdown(options=('All', 'COMEDIE MUSICALE', 'DANSE HIP-HOP', 'HARD METAL', 'POP ROCK FOLK', 'RAP HIP-HOP SLAM…

Label(value='Filter by Location:')

Dropdown(options=('All', 'ACCOR ARENA, PARIS', 'CASINO DE PARIS, PARIS', 'Dome de Paris - Palais des sports, P…